## installing kaggle library

In [33]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install kaggle

  Using cached kaggle-1.7.4.2-py3-none-any.whl.metadata (16 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
Using cached kaggle-1.7.4.2-py3-none-any.whl (173 kB)
Using cached bleach-6.2.0-py3-none-any.whl (163 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\kaggle.exe' -> 'c:\\Python312\\Scripts\\kaggle.exe.deleteme'


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Configuring the path of kaggle.json file

In [4]:
import os
import shutil

# Ensure .kaggle folder exists
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Move kaggle.json file to the correct location
shutil.copy('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))

# Set secure file permissions
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)


In [5]:
import os
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)


## Importing twitter sentiment dataset

In [15]:
# Install kagglehub if not already installed
%pip install kagglehub[pandas-datasets]



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Resuming download from 24117248 bytes (60738431 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/kazanova/sentiment140?dataset_version_number=2 (24117248/84855679) bytes left.


100%|██████████| 80.9M/80.9M [00:13<00:00, 4.47MB/s]

Extracting files...


Path to dataset files: C:\Users\hp\.cache\kagglehub\datasets\kazanova\sentiment140\versions\2


In [7]:
import kagglehub
import shutil
import os

# Download the dataset
path = kagglehub.dataset_download("kazanova/sentiment140")

# Define your desired path
desired_path = "./sentiment140"

# Move dataset to your desired location
if not os.path.exists(desired_path):
    shutil.move(path, desired_path)

print(f"Dataset moved to: {desired_path}")


Dataset moved to: ./sentiment140


## Import the dependencies

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Ensure nltk is imported correctly
import nltk

# Download stopwords if not already downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
## Printing the stopwords in English

print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## Data Processing

In [4]:
## Loading the data from CSV file to pandas dataframe
twitter_data = pd.read_csv('D:/Sentiment Analisis on Twitter and Reddit/sentiment140/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')


In [5]:
## Checking the number of columns and rows
twitter_data.shape

(1599999, 6)

In [6]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('D:/Sentiment Analisis on Twitter and Reddit/sentiment140/training.1600000.processed.noemoticon.csv', names=column_names ,encoding='ISO-8859-1')

In [8]:
twitter_data.shape

(1600000, 6)

In [9]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
## Counting the number of missing values in the dataset

twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [11]:
## checking the disturbuting of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

## Converting the target 4 to 1

## 0 --> Negative Tweet
## 1 --> Positive Tweet

In [12]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [13]:
## checking the disturbuting of target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

## Stemming

# Stemming is the process of reducing the word to its root word

example: actor , actress, acting = act

In [14]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    
    stemmed_content = re.sub('[^a-zA-Z]','',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ''.join(stemmed_content)

    return stemmed_content

In [17]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [18]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [19]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [20]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [21]:
# Seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values



In [22]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [23]:
print(Y)

[0 0 0 ... 1 1 1]


## Splitting the Data to Traning data and test Data

In [24]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

In [25]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [26]:
print(X_train)

['abouttowatchsawivanddrinkalilwin' 'hatermagazineimin'
 'eventhoughitsmyfavouritedrinkithinkitsthevodkaandcokethatwipesmymindallthetimethinkimgonnahavetofindanewdrink'
 ... 'iseagerformondayafternoon'
 'hopeeveryoneandtheirmotherhadagreatdaycantwaittohearwhattheguyshaveinstoretomorrow'
 'ilovewakinguptofolgerstoobadmyvoicewasdeeperthanhi']


In [27]:
print(X_test)

['mmangenmdoingfineihaventhadmuchtimetochatontwitterhubbyisbackforthesummeramptendstodominatemyfreetim'
 'atahsmayshowwruthkimampgeoffreysanhueza'
 'ishataramaybeitwasonlyabayareathangdammit' ...
 'destinineverthelesshoorayformembersandhaveawonderfulandsafetrip'
 'notfeelingtoowel' 'supersandrothanky']


In [28]:
# Converting the textual data into numberical data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test =  vectorizer.transform(X_test)

In [29]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1279968 stored elements and shape (1280000, 1254689)>
  Coords	Values
  (0, 4269)	1.0
  (1, 370478)	1.0
  (2, 259878)	1.0
  (3, 544662)	1.0
  (4, 1125692)	1.0
  (5, 183084)	1.0
  (6, 655943)	1.0
  (7, 427969)	1.0
  (8, 372293)	1.0
  (9, 414371)	1.0
  (10, 453206)	1.0
  (11, 946311)	1.0
  (12, 1165280)	1.0
  (13, 334893)	1.0
  (14, 1029544)	1.0
  (15, 1251389)	1.0
  (16, 26694)	1.0
  (17, 620642)	1.0
  (18, 371055)	1.0
  (19, 166384)	1.0
  (20, 939661)	1.0
  (21, 392755)	1.0
  (22, 288391)	1.0
  (23, 202689)	1.0
  (24, 1253472)	1.0
  :	:
  (1279975, 97244)	1.0
  (1279976, 1110776)	1.0
  (1279977, 938969)	1.0
  (1279978, 783480)	1.0
  (1279979, 800934)	1.0
  (1279980, 865418)	1.0
  (1279981, 587370)	1.0
  (1279982, 952975)	1.0
  (1279983, 412833)	1.0
  (1279984, 81684)	1.0
  (1279985, 59066)	1.0
  (1279986, 1015971)	1.0
  (1279987, 777243)	1.0
  (1279988, 1218709)	1.0
  (1279989, 450972)	1.0
  (1279990, 465151)	1.0
  (1279991,

In [30]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8600 stored elements and shape (320000, 1254689)>
  Coords	Values
  (22, 137709)	1.0
  (36, 790487)	1.0
  (55, 1232068)	1.0
  (104, 806213)	1.0
  (316, 675309)	1.0
  (317, 183110)	1.0
  (326, 530902)	1.0
  (343, 483231)	1.0
  (356, 1004786)	1.0
  (405, 294045)	1.0
  (411, 754846)	1.0
  (412, 671785)	1.0
  (503, 922733)	1.0
  (575, 484365)	1.0
  (576, 68315)	1.0
  (585, 553737)	1.0
  (597, 437993)	1.0
  (704, 437477)	1.0
  (730, 522522)	1.0
  (735, 1190472)	1.0
  (763, 201539)	1.0
  (851, 230654)	1.0
  (887, 945330)	1.0
  (905, 1080766)	1.0
  (965, 64114)	1.0
  :	:
  (319388, 469685)	1.0
  (319391, 363430)	1.0
  (319401, 69390)	1.0
  (319403, 1010168)	1.0
  (319419, 435823)	1.0
  (319442, 1154002)	1.0
  (319462, 331998)	1.0
  (319474, 363236)	1.0
  (319550, 1020181)	1.0
  (319573, 1087679)	1.0
  (319595, 377142)	1.0
  (319630, 416243)	1.0
  (319632, 324586)	1.0
  (319640, 333810)	1.0
  (319649, 564791)	1.0
  (319673, 473916)	

## Traning the Machine Learning Model

In [31]:
## Logistic Regression
model = LogisticRegression(max_iter=1000)


In [32]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

## Model Evaluation

Accuracy Score

In [33]:
## Accuracy Score on the training data
X_train_predication = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_predication)

In [34]:
print('Accuracy Score On The Training Data :',training_data_accuracy)

Accuracy Score On The Training Data : 0.99812265625


In [35]:
## Accuracy Score on the test data
X_test_predication = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_predication)

In [36]:
print('Accuracy Score On The Test Data :',test_data_accuracy)

Accuracy Score On The Test Data : 0.512653125


## Model Accuracy is 51%

Saving the Trained Model

In [37]:
import pickle

In [38]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

## Using the Saved Model For future Predication

In [41]:
loaded_model = pickle.load(open('D:/Sentiment Analisis on Twitter and Reddit/trained_model.sav','rb'))


In [44]:
X_new = X_test[200]
print(Y_test[200])

predication = model.predict(X_new)
print(predication)

if (predication[0] == 0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

1
[1]
Positive Tweet


In [50]:
X_new = X_test[3]
print(Y_test[3])

predication = loaded_model.predict(X_new)
print(predication)

if (predication[0] == 0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

0
[1]
Positive Tweet
